In [2]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter
s3 = var("s3", latex_name="s_3")        ## Third relaxation parameter

assume(C,'real')
assume(C>-1/2)
assume(C< 1/2)

s2 = 2
s3 = 2

#### Pieces involved in the scheme
q = 3 ## Number of discrete velocities
c1 = 0  ## First discrete velocity
c2 = 1  ## Second discrete velocity
c3 = -1 ## Third discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1,  1], \
            [0,  1, -1], \
            [0,  1,  1]]) ## Moment matrix
eps1 = 1              ## First equilibrium coefficient (useless)
eps2 = C              ## Second equilibrium coefficient
eps3 = 1/3*(1+2*C**2) ## Third equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa")
#assume(k>0)

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2, s3])\
        *(matrix([[eps1], [eps2], [eps3]])*matrix([[1, 0, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2), k**(-c3)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z")
#assume(z>0)

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = (dm1/dp1).full_simplify() ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [3]:
##### Boundary conditions that we analyze for any value of C and s_2 and s_3
bdMat_BB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 1], [0, 0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, -1], [0, 0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [-1, 0, -k], [0, 0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 1, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 2-k, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 3-3*k+k**2, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 4-6*k+4*k**2-k**3, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 0], [0, 0, k]])*Minv*K ## Kinetic dirichlet

In [4]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k))) 
print("Two steps anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [5]:
##### Sanity check : we set some values for C and see if we find more
##### Sometimes sagemath does not give all the roots

C_val = -3/13

print("Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_BB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_ABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots")
print("Two steps anti Bounce back")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_TwoABB.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma1.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma2.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma3.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_sigma4.subs(C=C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.subs(C=C_val).full_simplify(), bdMat_kinDir.subs(C=C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == -1], [z == -1, kappa == 1]]

Anti Bounce back


[[z == 0, kappa == 0], [z == -1, kappa == 1], [z == -16/507*I*sqrt(935) + 133/507, kappa == -1872/73529*I*sqrt(935) - 46151/73529], [z == 16/507*I*sqrt(935) + 133/507, kappa == 1872/73529*I*sqrt(935) - 46151/73529]]

!!!! We see that the previous computation for any value of C for the anti bounce back condition has missed a couple of complex conjugate roots
Two steps anti Bounce back


...

[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == -1, kappa == 1], [z == 1, kappa == 1], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -1, kappa == 1]]

In [6]:
print("We study possible blowups of the stable eigenvector close to the unit circle")
LzOnKplus = Lz.subs(k==kplus)
LzOnKminus = Lz.subs(k==kminus)

for i in range(q):
    for j in range(q):
        LzOnKplus[i, j]  = LzOnKplus[i, j].full_simplify()
        LzOnKminus[i, j] = LzOnKminus[i, j].full_simplify()
        
        
LzOnKplusKer = LzOnKplus.transpose().kernel()
LzOnKminusKer = LzOnKminus.transpose().kernel()

phiPlus = matrix([[LzOnKplusKer.basis()[0][0].full_simplify()], \
                  [LzOnKplusKer.basis()[0][1].full_simplify()], \
                  [LzOnKplusKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates

phiMinus = matrix([[LzOnKminusKer.basis()[0][0].full_simplify()], \
                   [LzOnKminusKer.basis()[0][1].full_simplify()], \
                   [LzOnKminusKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates

print(solve(phiPlus[1, 0].denominator().full_simplify(), z))
pretty_print(solve(phiPlus[2, 0].denominator().full_simplify(), z))

print(solve(phiMinus[1, 0].denominator().full_simplify(), z))
pretty_print(solve(phiMinus[2, 0].denominator().full_simplify(), z))

We study possible blowups of the stable eigenvector close to the unit circle
[
z == -1,
z == 1,
0 == 81*(6*C^2 + 3*C - 2)*z^13 + 81*z^14 + 27*(36*C^4 + 48*C^3 - 7*C^2 - 9*C + 1)*z^12 + 3*(232*C^6 + 732*C^5 + 366*C^4 + 21*C^3 - 24*C^2 - 78*C - 9*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9) + 20)*z^11 + 3*(32*C^8 + 384*C^7 + 632*C^6 + 612*C^5 + 396*C^4 - 189*C^3 - 125*C^2 - 3*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9)*(14*C^2 + 9*C - 5) + 30*C - 17)*z^10 + (512*C^8 + 1488*C^7 + 2824*C^6 + 1116*C^5 - 258*C^4 - 216*C^3 - 608*C^2 - 9*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9)*(20*C^4 + 36*C^3 - 3*C^2 - 6*C + 1) - 39*C + 122)*z^9 + (448*C^8 + 1776*C^7 + 2120*C^6 + 756*C^5 - 1470*C^4 - 1224*C^3 + 296*C^2 - 3*(24*C^6 + 124*C^5 + 70*C^4 + C^3 + 19*C^2 - 17*C - 5)*sqrt(6*(4*C^2 - 1)*z^

[z == -1]

[
z == -1,
z == 1,
0 == 81*(6*C^2 + 3*C - 2)*z^13 + 81*z^14 + 27*(36*C^4 + 48*C^3 - 7*C^2 - 9*C + 1)*z^12 + 3*(232*C^6 + 732*C^5 + 366*C^4 + 21*C^3 - 24*C^2 - 78*C + 9*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9) + 20)*z^11 + 3*(32*C^8 + 384*C^7 + 632*C^6 + 612*C^5 + 396*C^4 - 189*C^3 - 125*C^2 + 3*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9)*(14*C^2 + 9*C - 5) + 30*C - 17)*z^10 + (512*C^8 + 1488*C^7 + 2824*C^6 + 1116*C^5 - 258*C^4 - 216*C^3 - 608*C^2 + 9*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4*C^2 - 1)*z + 9)*(20*C^4 + 36*C^3 - 3*C^2 - 6*C + 1) - 39*C + 122)*z^9 + (448*C^8 + 1776*C^7 + 2120*C^6 + 756*C^5 - 1470*C^4 - 1224*C^3 + 296*C^2 + 3*(24*C^6 + 124*C^5 + 70*C^4 + C^3 + 19*C^2 - 17*C - 5)*sqrt(6*(4*C^2 - 1)*z^5 + 9*z^6 + 9*(4*C^2 - 1)*z^4 + 12*(2*C^2 + 1)*z^3 + 9*(4*C^2 - 1)*z^2 + 6*(4

[z == -1]

In [7]:
print("I do not know why assumptions on the sign of C must be done to obtain the right result")
print("Take C<0")
forget()
assume(C<0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print('Perturbation test')
dz = var("dz", latex_name="\\delta z") 
print('For kminus')
pretty_print(taylor(kminus.subs(z==-1-dz), dz, 0, 1))
print('For kplus')
pretty_print(taylor(kplus.subs(z==-1-dz), dz, 0, 1))
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus ### This is the right parametrization where we need it.

pretty_print(taylor(phiPlus[1, 0], z, 1, 0))
pretty_print(taylor(phiPlus[1, 0].denominator(), z, 1, 0))
pretty_print(taylor(phiPlus[1, 0].numerator(), z, 1, 0))

pretty_print(taylor(phiPlus[1, 0], z, -1, -1).factor())
pretty_print(taylor(phiPlus[1, 0].denominator(), z, -1, 0))
pretty_print(taylor(phiPlus[1, 0].numerator(), z, -1, 0))

pretty_print(taylor(phiPlus[2, 0], z, -1, 0))
pretty_print(taylor(phiPlus[2, 0].denominator(), z, -1, 0))
pretty_print(taylor(phiPlus[2, 0].numerator(), z, -1, 0))

print("Group velocity of the stable mode at z=1")
pretty_print((-ks/z/ks.derivative(z)).subs(z==1).full_simplify())

forget()
print("Take C>0")
assume(C>0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print('Perturbation test')
dz = var("dz", latex_name="\\delta z") 
print('For kminus')
pretty_print(taylor(kminus.subs(z==-1-dz), dz, 0, 1))
print('For kplus')
pretty_print(taylor(kplus.subs(z==-1-dz), dz, 0, 1))
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())


ks = kplus ### This is the right parametrization where we need it.

pretty_print(taylor(phiPlus[1, 0], z, 1, 0))
pretty_print(taylor(phiPlus[1, 0].denominator(), z, 1, 4))
pretty_print(taylor(phiPlus[1, 0].numerator(), z, 1, 4))

pretty_print(taylor(phiPlus[1, 0], z, -1, -1).factor())
pretty_print(taylor(phiPlus[1, 0].denominator(), z, -1, 2))
pretty_print(taylor(phiPlus[1, 0].numerator(), z, -1, 1))

pretty_print(taylor(phiPlus[2, 0], z, -1, 0))
pretty_print(taylor(phiPlus[2, 0].denominator(), z, -1, 2))
pretty_print(taylor(phiPlus[2, 0].numerator(), z, -1, 1))

print("Group velocity of the stable mode at z=1")
pretty_print((-ks/z/ks.derivative(z)).subs(z==1).full_simplify())

I do not know why assumptions on the sign of C must be done to obtain the right result
Take C<0
Value of kminus at z=-1


1

Value of kplus at z=-1


1

Perturbation test
For kminus


...

4/3*(C^2 - 1)/(z + 1) + 1

3*z + 3

4*C^2 + 3*z - 1

Group velocity of the stable mode at z=1


C

In [8]:
print("Anti Bounce back (now correct)")
kABB = solve(bdMat_ABB.determinant().full_simplify(), k)[0].rhs().full_simplify()
zABB = solve(charEq.full_simplify().subs(k==kABB), z)
pretty_print(zABB)

zABBMinus = zABB[0].rhs()
zABBPlus  = zABB[1].rhs()

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBMinus).full_simplify(), k))

pretty_print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBPlus).full_simplify(), k))
pretty_print(solve(charEq.subs(z==zABBPlus).full_simplify(), k))

#print(solve(bdMat_ABB.determinant().full_simplify().subs(z==zABBMinus).full_simplify(), k))

R = -(4*C^4 - 2*C^2 - 2).factor()
RealPoly = 32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - 6*C + 10
ImagPoly = 16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2
ER = (1/2*(RealPoly+RealPoly.subs(C==-C))).factor()
OR = (1/2*(RealPoly-RealPoly.subs(C==-C))).factor()
EI = (1/2*(ImagPoly+ImagPoly.subs(C==-C))).factor()
OI = (1/2*(ImagPoly-ImagPoly.subs(C==-C))).factor()

print('values of E_R, O_R, E_I and O_I')
pretty_print(ER)
pretty_print(OR)
pretty_print(EI)
pretty_print(OI)

er = var("er", latex_name="\\mathcal{E}_R") 
ei = var("ei", latex_name="\\mathcal{E}_I") 
odr = var("odr", latex_name="\\mathcal{O}_R") 
odi = var("odi", latex_name="\\mathcal{O}_I") 
r = var("r", latex_name="\\mathcal{R}") 
assume(er, 'real')
assume(ei, 'real')
assume(odr, 'real')
assume(odi, 'real')
assume(r>0)


num = (er+odr+I*(ei+odi)*sqrt(r))
den = (er-odr+I*(ei-odi)*sqrt(r))

numMod = ((num*conjugate(den)).full_simplify())
denMod = ((den*conjugate(den)).full_simplify())
print('Numerator in abs(k)^2')
pretty_print((numMod.real()**2+numMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())
print('Denominator in abs(k)^2')
pretty_print((denMod.real()**2+denMod.imag()**2).full_simplify().factor().subs(er==ER, ei==EI, odr==OR, odi==OI, r==R).full_simplify().factor())


dz = var("dz", latex_name="\\delta z") 
dk = var("dk", latex_name="\\delta \\kappa") 

print('Perturbation test on the other root (k=-1) on the critical z-s')
pretty_print(solve(taylor(charEq.subs(z==zABBMinus*(1+dz), k==-1+dk).full_simplify(), (dz, 0), (dk, 0), 1), dk))
pretty_print(solve(taylor(charEq.subs(z==zABBPlus*(1+dz), k==-1+dk).full_simplify(), (dz, 0), (dk, 0), 1), dk))

#assume(C<0)

print('To see whether the parametrization here is kminus or kplus')
for Cval in [-1/10, -1/4, -11/23]:
    pretty_print(kminus.subs(z==zABBMinus).full_simplify().subs(C==Cval).n())
    pretty_print(kplus.subs(z==zABBMinus).full_simplify().subs(C==Cval).n())
    pretty_print(kminus.subs(z==zABBPlus).full_simplify().subs(C==Cval).n())
    pretty_print(kplus.subs(z==zABBPlus).full_simplify().subs(C==Cval).n())
    print('---')
print('Conclusion: the right parametrization is kplus')    

ks = kplus.full_simplify()

Cval = -1/4
Vgroup = -(ks/z/derivative(ks, z)).subs(z==zABBMinus).full_simplify().subs(C==Cval) ### Group velocity without \lambda
print('group velocity (without lambda)')
pretty_print(Vgroup)
print('group velocity (without lambda), totally simplified')
pretty_print(abs(Vgroup.maxima_methods().polarform()).canonicalize_radical()) ### https://ask.sagemath.org/question/43617/is-there-a-sage-command-to-compute-complex-numbers/
#### This maxima_methods().polarform()).canonicalize_radical() can be in principle be used to study the modulus of k as done before

var('n')
print('To understand what is the resonant source term for this instability')
pretty_print((zABBMinus.canonicalize_radical().maxima_methods().polarform()**n).expand().full_simplify())
pretty_print((zABBPlus.canonicalize_radical().maxima_methods().polarform()**n).expand().full_simplify())


print('Computation of the stable eigenvector')
LzOnStable = Lz.subs(k==ks)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
LzOnStableKer = LzOnStable.transpose().kernel()
phiS = matrix([[LzOnStableKer.basis()[0][0].full_simplify()], \
               [LzOnStableKer.basis()[0][1].full_simplify()], \
               [LzOnStableKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates

print('We observe that the eigenvector does not blow up')
for Cval in [-1/10, -1/4, -11/23]:
    pretty_print(phiS[1][0].subs(z==zABBMinus).subs(C==Cval).canonicalize_radical().full_simplify().n())
    pretty_print(phiS[2][0].subs(z==zABBMinus).subs(C==Cval).canonicalize_radical().full_simplify().n())
    print('----')
    
print("I check that the eigenvector for the bulk is also eigenvector for the boundary, thus instability")
print("Computations are performed directly on the limit point")
kStar = kABB.subs(z==zABBMinus).full_simplify().canonicalize_radical()
LzOnStable = Lz.subs(z==zABBMinus.canonicalize_radical(), k==kStar)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
LzOnStableKer = LzOnStable.transpose().kernel()
phiS = matrix([[LzOnStableKer.basis()[0][0].full_simplify()], \
               [LzOnStableKer.basis()[0][1].full_simplify()], \
               [LzOnStableKer.basis()[0][2].full_simplify()]]) ### It can take a certain time but it terminates
bd00 = matrix(SR, q, q)
bd01 = matrix(SR, q, q)

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].subs(z==zABBMinus.canonicalize_radical()).full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].subs(z==zABBMinus.canonicalize_radical()).full_simplify().collect(k).coefficient(k, 1)
        
firstColBKL = (bd00 + bd01*kStar)*phiS ### First column
for i in range(q):
    pretty_print(firstColBKL[i][0].full_simplify())

Anti Bounce back (now correct)


[z == -4/3*C^2 - 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -4/3*C^2 + 2/3*sqrt(4*C^4 - 2*C^2 - 2) + 1/3, z == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 + (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 + (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10), kappa == -1]

[kappa == -(32*C^6 + 48*C^5 - 36*C^4 - 42*C^3 - 6*C^2 - (16*C^4 + 24*C^3 - 14*C^2 - 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) - 6*C + 10)/(32*C^6 - 48*C^5 - 36*C^4 + 42*C^3 - 6*C^2 - (16*C^4 - 24*C^3 - 14*C^2 + 15*C - 2)*sqrt(4*C^4 - 2*C^2 - 2) + 6*C + 10)]

...

0.485822306238185 - 0.499798952579531*I

----
I check that the eigenvector for the bulk is also eigenvector for the boundary, thus instability
Computations are performed directly on the limit point


0

0

0

In [9]:
### Critical points (-1, 1) and (1, -1) (plus (zstar, kstar) for the anti bounce back condition)
forget()
assume(C<0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print('Perturbation test')
dz = var("dz", latex_name="\\delta z") 
print('For kminus')
pretty_print(taylor(kminus.subs(z==-1-dz), dz, 0, 1))
print('For kplus')
pretty_print(taylor(kplus.subs(z==-1-dz), dz, 0, 1))
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus.full_simplify()

print("Group velocity of the stable mode at z=-1")
pretty_print((ks/z/(taylor(kplus.subs(z==-1-dz), dz, 0, 1) - 1).subs(dz==1)).subs(z==-1).full_simplify())

print("Group velocity of the stable mode at z=1")
pretty_print((-ks/z/ks.derivative(z)).subs(z==1).full_simplify())


LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()], \
               [LzOnStable.transpose().kernel().basis()[0][2].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = -1')
pretty_print(taylor(phiS[1][0], z, -1, -1).factor())
print('Expansion of the third component of phiS around z = -1')
pretty_print(taylor(phiS[2][0], z, -1, 0))
print('Expansion of the second component of phiS around z = 1')
pretty_print(taylor(phiS[1][0], z, 1, 1))
print('Expansion of the third component of phiS around z = 1')
pretty_print(taylor(phiS[2][0], z, 1, 1))



#### We construct the eigenvectors for k = 0
print("Basis for the eigenspace of k = 0")
pretty_print(Lz.subs(k==0).transpose().kernel().basis())
phi01 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][2]]]) ## First vector
phi02 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[1][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][2]]]) ## Second vector

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)
BKL = matrix(SR, q, q) #### Boundary matrix on the stable subspace


g1m1Tilde = 1 ### Source term used in the experiments

Value of kminus at z=-1


1

Value of kplus at z=-1


1

Perturbation test
For kminus


...

-1/6*(2*C^2 + 1)*(z - 1)/C - 2*C/(z - 1)

Expansion of the third component of phiS around z = 1


2/3*C^2 - 1/3*(C^2 - 1)*(z - 1) + 1/3

Basis for the eigenspace of k = 0


[
(1, 0, 4/3*C^2 + 2*C + 2/3),
(0, 1, -1)
]

In [9]:
print("======= Bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0))

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = -2/3*(2*C**2*z-2*C**2+3*C*z+3*C-2*z+2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]
m3 = Cs*z**(n-1)*ks**(j)*phiS[2, 0]


print("m1_j^n equals approximately")
pretty_print((m1.full_simplify().maxima_methods().residue(z,-1) \
            + m1.full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately")
pretty_print((m2.full_simplify().maxima_methods().residue(z,-1) \
            + m2.full_simplify().maxima_methods().residue(z,1)).factor())
print("m3_j^n equals approximately")
pretty_print((m3.full_simplify().maxima_methods().residue(z,-1) \
            + m3.full_simplify().maxima_methods().residue(z,1)).factor())

======= Bounce back ======
Kreiss Lopatinskii determinant at z = -1


-32/9*(5*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 + 3*C^5 - 10*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 30*C^3 - 4*I*sqrt(3)*sqrt(5*C^2 - 8) - 60*C)*(C + 1)^2*(C - 1)^2/((-3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 11*C^6 - 60*C^4 + 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 16)*(z + 1))

Kreiss Lopatinskii determinant at z = 1


0

Computation of the long-time solution for the source term in the experiments using residues
m1_j^n equals approximately


2*(-1)^j

m2_j^n equals approximately


-2/3*(4*C^2*j + 2*C^2 + 6*C*n - 6*C + 2*j + 1)*(-1)^j

m3_j^n equals approximately


2/3*(2*C^2 + 1)*(-1)^j

In [14]:
print("======= Anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, -1).factor())
print("Kreiss Lopatinskii determinant at z = zStarPlus")
print('First column of BKL')
pretty_print(BKL[:, 0].subs(z==zABBPlus).simplify_full())
print('It is not super easy for general C to see that this vanishes')
for Cval in [-1/4, -1/3, -13/87]:
    print('C = '+str(Cval))
    pretty_print(BKL[:, 0].subs(z==zABBPlus).simplify_full().subs(C==Cval).simplify_full())

======= Anti bounce back ======
Kreiss Lopatinskii determinant at z = -1


32/9*(C + 1)^2*(C - 1)^2/(z + 1)

Kreiss Lopatinskii determinant at z = 1


8*C^2/(z - 1)

Kreiss Lopatinskii determinant at z = zStarPlus
First column of BKL


...

[0]
[0]
[0]

C = -1/3


[0]
[0]
[0]

C = -13/87


[0]
[0]
[0]

In [35]:
print("======= Two-steps anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 0).factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())

print("Ad hoc analysis of 1/(1-abs(ks)^2) in the vicinity of z=-1")
pretty_print(taylor(1/(1-abs(ks)**2), z, -1, -1))


print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = -2/3*(2*C**2*z-2*C**2+3*C*z+3*C-2*z+2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]
m3 = Cs*z**(n-1)*ks**(j)*phiS[2, 0]


print("m1_j^n equals approximately")
pretty_print((m1.full_simplify().maxima_methods().residue(z,-1) \
            + m1.full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately")
pretty_print((m2.full_simplify().maxima_methods().residue(z,-1) \
            + m2.full_simplify().maxima_methods().residue(z,1)).factor())
print("m3_j^n equals approximately")
pretty_print((m3.full_simplify().maxima_methods().residue(z,-1) \
            + m3.full_simplify().maxima_methods().residue(z,1)).factor())

======= Two-steps snti bounce back ======
Kreiss Lopatinskii determinant at z = -1


8/3*(C + 1)*(C - 1)

Kreiss Lopatinskii determinant at z = 1


-4*C

Ad hoc analysis of 1/(1-abs(ks)^2) in the vicinity of z=-1


...

0

m2_j^n equals approximately


-2*(3*I*sqrt(3)*sqrt(5*C^2 - 8)*(-1)^j*C^2 - 2*I*sqrt(3)*sqrt(5*C^2 - 8)*(-1)^n*C^2 - 3*(-1)^j*C^3 - 6*(-1)^n*C^3 + 2*I*sqrt(3)*sqrt(5*C^2 - 8)*(-1)^n + 12*(-1)^j*C + 6*(-1)^n*C)/(3*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 3*C^2 + 12)

m3_j^n equals approximately


4/3*(-1)^n*(C + 1)*(C - 1)

In [43]:
print("======= Extrapolation sigma = 1 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 0).full_simplify().factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = -2/3*(2*C**2*z-2*C**2+3*C*z+3*C-2*z+2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]
m3 = Cs*z**(n-1)*ks**(j)*phiS[2, 0]


print("m1_j^n equals approximately")
pretty_print((m1.full_simplify().maxima_methods().residue(z,-1) \
            + m1.full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately")
pretty_print((m2.full_simplify().maxima_methods().residue(z,-1) \
            + m2.full_simplify().maxima_methods().residue(z,1)).factor())
print("m3_j^n equals approximately")
pretty_print((m3.full_simplify().maxima_methods().residue(z,-1) \
            + m3.full_simplify().maxima_methods().residue(z,1)).factor())

======= Extrapolation sigma = 1 ======
Kreiss Lopatinskii determinant at z = -1


-8*(I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 39*C^6 - 10*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 6*C^5 - 20*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 - 120*C^4 + 20*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 60*C^3 + 28*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 8*I*sqrt(3)*sqrt(5*C^2 - 8) + 120*C + 48)*(C + 1)^2*(C - 1)/(27*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 99*C^6 + 540*C^4 - 108*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 540*C^2 - 144)

Kreiss Lopatinskii determinant at z = 1


4/3*(2*C^2*z - 2*C^2 + 15*C*z - 9*C - 2*z + 2)*C/(z - 1)

Computation of the long-time solution for the source term in the experiments using residues
m1_j^n equals approximately


0

m2_j^n equals approximately


4*(I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 39*C^7 - 30*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 126*C^5 + 48*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 8*I*sqrt(3)*sqrt(5*C^2 - 8) + 168*C)*(-1)^n*(C - 1)/(19*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7 - 27*C^8 + 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 78*C^7 - 48*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 300*C^6 - 60*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 252*C^5 - 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 - 720*C^4 + 96*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 80*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 432*C^2 + 16*I*sqrt(3)*sqrt(5*C^2 - 8) + 336*C + 96)

m3_j^n equals approximately


-4*(3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 11*C^6 + 60*C^4 - 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 60*C^2 - 16)*(-1)^n*(C - 1)/(I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 39*C^6 - 10*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 6*C^5 - 20*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 - 120*C^4 + 20*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 60*C^3 + 28*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 8*I*sqrt(3)*sqrt(5*C^2 - 8) + 120*C + 48)

In [48]:
print("======= Extrapolation sigma = 2 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 1).full_simplify().factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())


print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = -2/3*(2*C**2*z-2*C**2+3*C*z+3*C-2*z+2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]
m3 = Cs*z**(n-1)*ks**(j)*phiS[2, 0]


print("m1_j^n equals approximately")
pretty_print((m1.full_simplify().maxima_methods().residue(z,-1) \
            + m1.full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately")
pretty_print((m2.full_simplify().maxima_methods().residue(z,-1) \
            + m2.full_simplify().maxima_methods().residue(z,1)).factor())
print("m3_j^n equals approximately")
pretty_print((m3.full_simplify().maxima_methods().residue(z,-1) \
            + m3.full_simplify().maxima_methods().residue(z,1)).factor())

======= Extrapolation sigma = 2 ======
Kreiss Lopatinskii determinant at z = -1


-4*(7*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 17*C^7 - 6*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 22*C^6 - 30*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 6*C^5 - 120*C^4 + 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 120*C^3 + 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 120*C^2 + 8*I*sqrt(3)*sqrt(5*C^2 - 8) + 136*C + 32)*(C + 1)*(z + 1)/(9*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 33*C^6 + 180*C^4 - 36*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 180*C^2 - 48)

Kreiss Lopatinskii determinant at z = 1


8/3*(4*C^2*z - 4*C^2 + 15*C*z - 9*C - z + 1)*C/(z - 1)

Computation of the long-time solution for the source term in the experiments using residues
m1_j^n equals approximately


-2*(3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 11*C^6 + 60*C^4 - 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 60*C^2 - 16)*(-1)^n*(C - 1)/(7*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 17*C^7 - 6*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 22*C^6 - 30*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 6*C^5 - 120*C^4 + 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 120*C^3 + 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 120*C^2 + 8*I*sqrt(3)*sqrt(5*C^2 - 8) + 136*C + 32)

m2_j^n equals approximately


2*(20444*I*sqrt(3)*sqrt(5*C^2 - 8)*C^26*n - 139644*C^27*n - 433*I*sqrt(3)*sqrt(5*C^2 - 8)*C^26 + 160521*C^27 - 19578*I*sqrt(3)*sqrt(5*C^2 - 8)*C^25*j - 181398*C^26*j + 59600*I*sqrt(3)*sqrt(5*C^2 - 8)*C^25*n + 223152*C^26*n - 70255*I*sqrt(3)*sqrt(5*C^2 - 8)*C^25 + 7191*C^26 + 100488*I*sqrt(3)*sqrt(5*C^2 - 8)*C^24*j - 56136*C^25*j + 2012*I*sqrt(3)*sqrt(5*C^2 - 8)*C^24*n + 2713524*C^25*n - 381406*I*sqrt(3)*sqrt(5*C^2 - 8)*C^24 - 2313126*C^25 + 660312*I*sqrt(3)*sqrt(5*C^2 - 8)*C^23*j + 1968864*C^24*j - 1244112*I*sqrt(3)*sqrt(5*C^2 - 8)*C^23*n - 945264*C^24*n + 826356*I*sqrt(3)*sqrt(5*C^2 - 8)*C^23 - 2640768*C^24 - 1068912*I*sqrt(3)*sqrt(5*C^2 - 8)*C^22*j + 4257936*C^23*j - 2660328*I*sqrt(3)*sqrt(5*C^2 - 8)*C^22*n - 18699336*C^23*n + 4869504*I*sqrt(3)*sqrt(5*C^2 - 8)*C^22 + 8254368*C^23 - 6009768*I*sqrt(3)*sqrt(5*C^2 - 8)*C^21*j - 2067336*C^22*j + 7790208*I*sqrt(3)*sqrt(5*C^2 - 8)*C^21*n - 15257664*C^22*n - 1369884*I*sqrt(3)*sqrt(5*C^2 - 8)*C^21 + 30594132*C^22 + 959328*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20*j - 43863264*C^21*j + 21277552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20*n + 50259024*C^21*n - 22375016*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20 + 26865048*C^21 + 22513152*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19*j - 60125856*C^20*j - 15028352*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19*n + 145076736*C^20*n - 16721024*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19 - 126496368*C^20 + 25957248*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18*j + 168041856*C^19*j - 72626048*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18*n + 31724544*C^19*n + 35347024*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18 - 281723472*C^19 - 24025248*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17*j + 363680544*C^18*j - 31479552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17*n - 497612544*C^18*n + 88780176*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17 + 155610672*C^18 - 122055552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16*j - 177289344*C^17*j + 108067392*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16*n - 559711296*C^17*n + 48445344*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16 + 817056288*C^17 - 82902528*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15*j - 897111936*C^16*j + 191720448*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15*n + 699480576*C^16*n - 152711424*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15 + 379069632*C^16 + 196604928*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14*j - 560507904*C^15*j + 12559872*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14*n + 1492176384*C^15*n - 254363136*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14 - 877496832*C^15 + 299561472*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13*j + 823325184*C^14*j - 317571072*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13*n + 138940416*C^14*n + 12423936*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13 - 1423588608*C^14 - 6838272*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12*j + 1884911616*C^13*j - 273988608*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12*n - 1687458816*C^13*n + 304310784*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12 - 429000192*C^13 - 327794688*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11*j + 660547584*C^12*j + 142368768*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11*n - 1694121984*C^12*n + 254509056*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11 + 1429281792*C^12 - 323592192*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10*j - 1824989184*C^11*j + 344881152*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10*n + 287483904*C^11*n - 12013056*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10 + 1799290368*C^11 + 2737152*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9*j - 2061075456*C^10*j + 167067648*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9*n + 1907490816*C^10*n - 224193024*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9 + 160123392*C^10 + 278581248*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8*j - 166662144*C^9*j - 100610048*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8*n + 1170284544*C^9*n - 198910976*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8 - 1133417472*C^9 + 219850752*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7*j + 1263212544*C^8*j - 185495552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7*n - 443756544*C^8*n - 31437824*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7 - 1008995328*C^8 + 28520448*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6*j + 1198534656*C^7*j - 83376128*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6*n - 928536576*C^7*n + 64835584*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 - 272904192*C^7 - 74815488*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5*j + 275810304*C^6*j + 18382848*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5*n - 457949184*C^6*n + 64306176*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 254487552*C^6 - 72179712*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4*j - 326836224*C^5*j + 39555

m3_j^n equals approximately


-2*(3956*I*sqrt(3)*sqrt(5*C^2 - 8)*C^21*n + 5804*C^22*n - 3956*I*sqrt(3)*sqrt(5*C^2 - 8)*C^21 - 5804*C^22 + 4418*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20*j - 10482*C^21*j - 4880*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20*n + 26768*C^21*n + 2671*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20 - 21527*C^21 + 3032*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19*j + 38376*C^20*j - 54932*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19*n + 21636*C^20*n + 50449*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19 - 45177*C^20 - 31372*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18*j + 235476*C^19*j + 1712*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18*n - 415856*C^19*n + 17634*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18 + 278042*C^19 - 99648*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17*j - 309216*C^18*j + 263520*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17*n - 797264*C^18*n - 175464*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17 + 931292*C^18 - 33048*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16*j - 1680264*C^17*j + 318816*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16*n + 2045664*C^17*n - 299916*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16 - 913716*C^17 + 650880*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15*j + 39744*C^16*j - 403584*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15*n + 4935648*C^16*n - 81264*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15 - 4378152*C^16 + 898176*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14*j + 5116032*C^15*j - 1693056*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14*n - 3129216*C^15*n + 1007232*I*sqrt(3)*sqrt(5*C^2 - 8)*C^14 - 671232*C^15 - 1401216*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13*j + 5954688*C^14*j - 799104*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13*n - 13753728*C^14*n + 1642992*I*sqrt(3)*sqrt(5*C^2 - 8)*C^13 + 7652016*C^14 - 3139584*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12*j - 4624128*C^13*j + 3310464*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12*n - 5215872*C^13*n - 707616*I*sqrt(3)*sqrt(5*C^2 - 8)*C^12 + 8632416*C^13 - 222336*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11*j - 20529792*C^12*j + 3920832*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11*n + 17267136*C^12*n - 3067056*I*sqrt(3)*sqrt(5*C^2 - 8)*C^11 + 188688*C^12 + 4117824*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10*j - 11171520*C^11*j - 1679616*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10*n + 24581376*C^11*n - 1829088*I*sqrt(3)*sqrt(5*C^2 - 8)*C^10 - 13979232*C^11 + 4663296*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9*j + 24067584*C^10*j - 5385216*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9*n - 1327872*C^10*n + 855552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^9 - 16465344*C^10 - 253056*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8*j + 31552128*C^9*j - 2747904*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8*n - 30586368*C^9*n + 2684352*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8 + 1390656*C^9 - 5044224*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7*j + 1354752*C^8*j + 2067456*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7*n - 20316672*C^8*n + 2295552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7 + 14875776*C^8 - 3781632*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6*j - 23749632*C^7*j + 3563520*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6*n + 9633792*C^7*n + 198144*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 11761152*C^7 + 36864*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5*j - 21270528*C^6*j + 1354752*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5*n + 18416640*C^6*n - 1082880*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 2692608*C^6 + 1838592*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4*j - 2612736*C^5*j - 663552*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4*n + 7667712*C^5*n - 1057536*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 4066560*C^5 + 1456128*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3*j + 8349696*C^4*j - 877568*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3*n - 2456576*C^4*n - 576256*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 - 5056768*C^4 + 566272*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2*j + 6964224*C^3*j - 397312*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2*n - 4173824*C^3*n - 189952*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 2764288*C^3 + 114688*I*sqrt(3)*sqrt(5*C^2 - 8)*C*j + 2727936*C^2*j - 90112*I*sqrt(3)*sqrt(5*C^2 - 8)*C*n - 1953792*C^2*n - 34816*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 906240*C^2 + 10240*I*sqrt(3)*sqrt(5*C^2 - 8)*j + 571392*C*j - 8192*I*sqrt(3)*sqrt(5*C^2 - 8)*n - 434176*C*n - 3072*I*sqrt(3)*sqrt(5*C^2 - 8) - 177152*C + 49152*j - 40960*n - 14336)*(-1)^n*(C - 1)/(693*I*sqrt(3)*sqrt(5*C^2 - 8)*C^20 - 24429*C^21 + 17802*I*sqrt(3)*sqrt(5*C^2 - 8)*C^19 + 26118*C^20 + 22086*I*sqrt(3)*sqrt(5*C^2 - 8)*C^18 + 352206*C^19 - 221184*I*sqrt(3)*sqrt(5*C^2 - 8)*C^17 + 160968*C^18 - 373140*I*sqrt(3)*sqrt(5*C^2 - 8)*C^16 - 1949292*C^17 + 821664*I*sqrt(3)*sqrt(5*C^2 - 8)*C^15 - 3579

In [49]:
print("======= Extrapolation sigma = 3 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 2).full_simplify().factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())


======= Extrapolation sigma = 3 ======
Kreiss Lopatinskii determinant at z = -1


2/3*(-19*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7 + 27*C^8 - 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 - 78*C^7 + 48*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 300*C^6 + 60*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 + 252*C^5 + 24*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 + 720*C^4 - 96*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 80*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 432*C^2 - 16*I*sqrt(3)*sqrt(5*C^2 - 8) - 336*C - 96)*(z + 1)^2/((3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 11*C^6 + 60*C^4 - 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 60*C^2 - 16)*(C - 1))

Kreiss Lopatinskii determinant at z = 1


16*(2*C*z - 2*C + 5*z - 3)*C^2/(z - 1)

In [50]:
print("======= Extrapolation sigma = 4 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2 + bd03*ks**3)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 3).full_simplify().factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())


======= Extrapolation sigma = 4 ======
Kreiss Lopatinskii determinant at z = -1


-(5*I*sqrt(3)*sqrt(5*C^2 - 8)*C^8 - 61*C^9 + 14*I*sqrt(3)*sqrt(5*C^2 - 8)*C^7 + 34*C^8 + 26*I*sqrt(3)*sqrt(5*C^2 - 8)*C^6 + 346*C^7 - 72*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 32*C^6 - 132*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 492*C^5 + 48*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 - 480*C^4 + 112*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 - 80*C^3 + 64*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 512*C^2 + 16*I*sqrt(3)*sqrt(5*C^2 - 8) + 368*C + 64)*(z + 1)^3/((3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 - 11*C^6 + 60*C^4 - 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 60*C^2 - 16)*(C + 1)*(C - 1)^2)

Kreiss Lopatinskii determinant at z = 1


32/3*(8*C^2*z - 8*C^2 + 15*C*z - 9*C + z - 1)*C/(z - 1)

In [10]:
print("======= Kinetic Dirichlet ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).full_simplify().factor())
print("Kreiss Lopatinskii determinant at z = 1")
pretty_print(taylor(DetKL, z, 1, 0).factor())

print("Computation of the long-time solution for the source term in the experiments using residues")
var('j')
var('n')

Cs = -2/3*(2*C**2*z-2*C**2+3*C*z+3*C-2*z+2)*z*g1m1Tilde/DetKL

m1 = Cs*z**(n-1)*ks**(j)*phiS[0, 0]
m2 = Cs*z**(n-1)*ks**(j)*phiS[1, 0]
m3 = Cs*z**(n-1)*ks**(j)*phiS[2, 0]


print("m1_j^n equals approximately")
pretty_print((m1.full_simplify().maxima_methods().residue(z,-1) \
            + m1.full_simplify().maxima_methods().residue(z,1)).factor())
print("m2_j^n equals approximately")
pretty_print((m2.full_simplify().maxima_methods().residue(z,-1) \
            + m2.full_simplify().maxima_methods().residue(z,1)).factor())
print("m3_j^n equals approximately")
pretty_print((m3.full_simplify().maxima_methods().residue(z,-1) \
            + m3.full_simplify().maxima_methods().residue(z,1)).factor())

======= Kinetic Dirichlet ======
Kreiss Lopatinskii determinant at z = -1


-16/9*(3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 11*C^5 + 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 + 14*C^4 - 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 46*C^3 - 8*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 16*C^2 - 4*I*sqrt(3)*sqrt(5*C^2 - 8) - 44*C - 16)*(C + 1)^3*(C - 1)^2/((-3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 11*C^6 - 60*C^4 + 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 16)*(z + 1))

Kreiss Lopatinskii determinant at z = 1


2*(5*C*z - 3*C - z + 1)*C/(z - 1)

Computation of the long-time solution for the source term in the experiments using residues
m1_j^n equals approximately


0

m2_j^n equals approximately


0

m3_j^n equals approximately


0

In [12]:
### Critical point (1, 1) and blowup point (-1, -1) for the stable eigenvector
forget()
assume(C>0)
print("Value of kminus at z=-1")
pretty_print(kminus.subs(z==-1).full_simplify())
print("Value of kplus at z=-1")
pretty_print(kplus.subs(z==-1).full_simplify())
print('Perturbation test')
dz = var("dz", latex_name="\\delta z") 
print('For kminus')
pretty_print(taylor(kminus.subs(z==-1-dz), dz, 0, 1))
print('For kplus')
pretty_print(taylor(kplus.subs(z==-1-dz), dz, 0, 1))
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).full_simplify())
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).full_simplify())

ks = kplus.full_simplify()

print("Group velocity of the stable mode at z=-1")
pretty_print((ks/z/(taylor(kplus.subs(z==-1-dz), dz, 0, 1) - 1).subs(dz==1)).subs(z==-1).full_simplify())

print("Group velocity of the stable mode at z=1")
pretty_print((-ks/z/ks.derivative(z)).subs(z==1).full_simplify())


LzOnStable = Lz.subs(k==ks) ### This is the matrix polynomial where k is replaced by ks (stable root)
for i in range(q):
    for j in range(q):
        LzOnStable[i, j] = LzOnStable[i, j].full_simplify()
        
#pretty_print(LzOnStable.transpose().kernel()) #### This is the stable eigenvalue phi_s
phiS = matrix([[LzOnStable.transpose().kernel().basis()[0][0]], \
               [LzOnStable.transpose().kernel().basis()[0][1].full_simplify()], \
               [LzOnStable.transpose().kernel().basis()[0][2].full_simplify()]])

#### Structure of the eigenvector close to the critical points
print('Expansion of the second component of phiS around z = -1')
pretty_print(taylor(phiS[1][0], z, -1, -1).factor())
print('Expansion of the third component of phiS around z = -1')
pretty_print(taylor(phiS[2][0], z, -1, 0))
print('Expansion of the second component of phiS around z = 1')
pretty_print(taylor(phiS[1][0], z, 1, 1))
print('Expansion of the third component of phiS around z = 1')
pretty_print(taylor(phiS[2][0], z, 1, 1))



#### We construct the eigenvectors for k = 0
print("Basis for the eigenspace of k = 0")
pretty_print(Lz.subs(k==0).transpose().kernel().basis())
phi01 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[0][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[0][2]]]) ## First vector
phi02 = matrix([[Lz.subs(k==0).transpose().kernel().basis()[1][0]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][1]], \
                [Lz.subs(k==0).transpose().kernel().basis()[1][2]]]) ## Second vector

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)
BKL = matrix(SR, q, q) #### Boundary matrix on the stable subspace


g1m1Tilde = 1 ### Source term used in the experiments

Value of kminus at z=-1


1

Value of kplus at z=-1


1

Perturbation test
For kminus


...

C - 1/6*(C^2 - 1)*(z - 1)/C

Expansion of the third component of phiS around z = 1


2/3*C^2 - 1/3*(C^2 - 1)*(z - 1) + 1/3

Basis for the eigenspace of k = 0


[
(1, 0, 4/3*C^2 + 2*C + 2/3),
(0, 1, -1)
]

In [13]:
print("======= Bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).factor())

======= Bounce back ======
Kreiss Lopatinskii determinant at z = -1


-32/9*(5*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 + 3*C^5 - 10*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 30*C^3 - 4*I*sqrt(3)*sqrt(5*C^2 - 8) - 60*C)*(C + 1)^2*(C - 1)^2/((-3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 11*C^6 - 60*C^4 + 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 16)*(z + 1))

In [14]:
print("======= Anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_ABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).factor())

======= Anti bounce back ======
Kreiss Lopatinskii determinant at z = -1


32/9*(C + 1)^2*(C - 1)^2/(z + 1)

In [26]:
print("======= Two-steps anti bounce back ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_TwoABB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, 0).factor())


======= Two-steps anti bounce back ======
Kreiss Lopatinskii determinant at z = -1


8/3*(C + 1)*(C - 1)

In [37]:
print("======= Extrapolation sigma = 1 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

BKL = BKL.simplify_full()

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
print("It is easier for Sage to show the first column of BKL")
pretty_print(taylor(BKL[:, 0], z, 1, 1))


======= Extrapolation sigma = 1 ======
Kreiss Lopatinskii determinant at z = 1
It is easier for Sage to show the first column of BKL


[1/6*(2*C^2 + 3*C + 1)*(z - 1)/C]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)/C]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)/C]

In [38]:
print("======= Extrapolation sigma = 2 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

BKL = BKL.simplify_full()

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
print("It is easier for Sage to show the first column of BKL")
pretty_print(taylor(BKL[:, 0], z, 1, 2))


======= Extrapolation sigma = 2 ======
Kreiss Lopatinskii determinant at z = 1
It is easier for Sage to show the first column of BKL


[1/6*(2*C^2 + 3*C + 1)*(z - 1)^2/C^2]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^2/C^2]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^2/C^2]

In [41]:
print("======= Extrapolation sigma = 3 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

BKL = BKL.simplify_full()

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
print("It is easier for Sage to show the first column of BKL")
pretty_print(taylor(BKL[:, 0], z, 1, 3))


======= Extrapolation sigma = 3 ======
Kreiss Lopatinskii determinant at z = 1
It is easier for Sage to show the first column of BKL


[1/6*(2*C^2 + 3*C + 1)*(z - 1)^3/C^3]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^3/C^3]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^3/C^3]

In [42]:
print("======= Extrapolation sigma = 4 ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
BKL[:, 0] = (bd00 + bd01*ks + bd02*ks**2 + bd03*ks**3)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

BKL = BKL.simplify_full()

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = 1")
print("It is easier for Sage to show the first column of BKL")
pretty_print(taylor(BKL[:, 0], z, 1, 4))


======= Extrapolation sigma = 4 ======
Kreiss Lopatinskii determinant at z = 1
It is easier for Sage to show the first column of BKL


[1/6*(2*C^2 + 3*C + 1)*(z - 1)^4/C^4]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^4/C^4]
[1/6*(2*C^2 + 3*C + 1)*(z - 1)^4/C^4]

In [45]:
print("======= Kinetic Dirichlet ======")

for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_kinDir[i, j].full_simplify().collect(k).coefficient(k, 1)
        
BKL[:, 0] = (bd00 + bd01*ks)*phiS ### First column
BKL[:, 1] = (bd00)*phi01 ### Second column
BKL[:, 2] = (bd00)*phi02 ### Third column

DetKL = BKL.determinant().full_simplify()

print("Kreiss Lopatinskii determinant at z = -1")
pretty_print(taylor(DetKL, z, -1, -1).factor())


======= Kinetic Dirichlet ======
Kreiss Lopatinskii determinant at z = -1


-16/9*(3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^4 - 11*C^5 + 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^3 + 14*C^4 - 2*I*sqrt(3)*sqrt(5*C^2 - 8)*C^2 + 46*C^3 - 8*I*sqrt(3)*sqrt(5*C^2 - 8)*C - 16*C^2 - 4*I*sqrt(3)*sqrt(5*C^2 - 8) - 44*C - 16)*(C + 1)^3*(C - 1)^2/((-3*I*sqrt(3)*sqrt(5*C^2 - 8)*C^5 + 11*C^6 - 60*C^4 + 12*I*sqrt(3)*sqrt(5*C^2 - 8)*C + 60*C^2 + 16)*(z + 1))